In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
from collections import Counter
import time
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
# 设置随机种子以确保结果可重现
torch.manual_seed(42)
np.random.seed(42)

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

使用设备: cuda


In [42]:
# class SentimentLSTM(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
#                  dropout, embedding_weights=None, train_embedding=True):
#         super(SentimentLSTM, self).__init__()
#
#         self.hidden_dim = hidden_dim
#         self.n_layers = n_layers
#
#         # 嵌入层
#         if embedding_weights is not None:
#             self.embedding = nn.Embedding.from_pretrained(embedding_weights, freeze=not train_embedding)
#         else:
#             self.embedding = nn.Embedding(vocab_size, embedding_dim)
#
#         # LSTM层
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
#                            dropout=dropout, batch_first=True, bidirectional=False)
#
#         # Dropout层
#         self.dropout = nn.Dropout(dropout)
#
#         # 全连接层
#         self.fc = nn.Linear(hidden_dim, output_dim)
#
#         # Sigmoid激活函数
#         self.sigmoid = nn.Sigmoid()
#
#     def forward(self, x):
#         # 嵌入层
#         embedded = self.embedding(x)
#
#         # LSTM层
#         lstm_out, (hidden, cell) = self.lstm(embedded)
#
#         # 取最后一个时间步的输出
#         out = self.dropout(lstm_out[:, -1, :])
#
#         # 全连接层
#         out = self.fc(out)
#
#         # Sigmoid激活
#         out = self.sigmoid(out)
#
#         return out.squeeze()

In [43]:
# import json
# import os
#
#
# class SentimentAnalyzer:
#     def __init__(self, device='cpu'):
#         self.device = device
#         self.vocab_size = 0
#         self.model = None
#
#     def create_model(self, vocab_size, embedding_dim=100, hidden_dim=128, n_layers=2,
#                     dropout=0.3, use_pretrained=False, glove_path=None, train_embedding=True):
#         """创建LSTM模型"""
#         self.vocab_size = vocab_size
#
#         if use_pretrained and glove_path:
#             embedding_weights = self.load_glove_embeddings(glove_path, embedding_dim, vocab_size)
#             if embedding_weights is not None:
#                 self.model = SentimentLSTM(
#                     vocab_size, embedding_dim, hidden_dim, 1, n_layers,
#                     dropout, embedding_weights, train_embedding
#                 )
#                 print("使用预训练GloVe词向量的模型已创建")
#             else:
#                 print("无法加载预训练词向量，使用随机初始化")
#                 use_pretrained = False
#
#         if not use_pretrained:
#             self.model = SentimentLSTM(
#                 vocab_size, embedding_dim, hidden_dim, 1, n_layers,
#                 dropout, None, True
#             )
#             print("使用随机初始化词向量的模型已创建")
#
#         self.model = self.model.to(self.device)
#         return self.model
#
#     def load_glove_embeddings(self, glove_path, embedding_dim, vocab_size):
#         """加载预训练的GloVe词向量"""
#         print("加载GloVe词向量...")
#
#         # 由于我们不知道确切的词汇表，我们假设索引0-29999对应词汇表中的词
#         embedding_weights = np.random.normal(scale=0.6, size=(vocab_size, embedding_dim))
#
#         try:
#             glove_embeddings = {}
#             with open(glove_path, 'r', encoding='utf-8') as f:
#                 for line in f:
#                     values = line.split()
#                     word = values[0]
#                     vector = np.asarray(values[1:], dtype='float32')
#                     glove_embeddings[word] = vector
#
#             print("注意: 由于缺乏索引到词的映射，无法精确加载GloVe词向量")
#             print("使用随机初始化替代")
#
#         except FileNotFoundError:
#             print(f"GloVe文件未找到: {glove_path}")
#             print("请从 https://nlp.stanford.edu/projects/glove/ 下载GloVe词向量")
#             return None
#
#         return torch.tensor(embedding_weights, dtype=torch.float32)
#
#     def train_model(self, train_loader, val_loader, num_epochs=10, learning_rate=0.001):
#         """训练模型"""
#         criterion = nn.BCELoss()
#         optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
#
#         train_losses = []
#         val_losses = []
#         train_accuracies = []
#         val_accuracies = []
#
#         best_val_loss = float('inf')
#         patience = 3
#         patience_counter = 0
#
#         for epoch in range(num_epochs):
#             start_time = time.time()
#
#             # 训练阶段
#             self.model.train()
#             train_loss = 0.0
#             train_correct = 0
#             train_total = 0
#
#             for batch_idx, (sequences, masks, labels, lengths) in enumerate(train_loader):
#                 sequences = sequences.to(self.device)
#                 labels = labels.to(self.device).float()
#
#                 optimizer.zero_grad()
#                 outputs = self.model(sequences)  # 不再传递lengths参数
#                 loss = criterion(outputs, labels)
#                 loss.backward()
#                 optimizer.step()
#
#                 train_loss += loss.item()
#                 predicted = (outputs > 0.5).float()
#                 train_total += labels.size(0)
#                 train_correct += (predicted == labels).sum().item()
#
#                 # 每100个batch打印一次进度
#                 if batch_idx % 100 == 0:
#                     print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")
#
#             # 验证阶段
#             self.model.eval()
#             val_loss = 0.0
#             val_correct = 0
#             val_total = 0
#
#             with torch.no_grad():
#                 for sequences, masks, labels, lengths in val_loader:
#                     sequences = sequences.to(self.device)
#                     labels = labels.to(self.device).float()
#
#                     outputs = self.model(sequences)  # 不再传递lengths参数
#                     loss = criterion(outputs, labels)
#
#                     val_loss += loss.item()
#                     predicted = (outputs > 0.5).float()
#                     val_total += labels.size(0)
#                     val_correct += (predicted == labels).sum().item()
#
#             # 计算平均损失和准确率
#             train_loss_avg = train_loss / len(train_loader)
#             val_loss_avg = val_loss / len(val_loader)
#             train_accuracy = train_correct / train_total
#             val_accuracy = val_correct / val_total
#
#             train_losses.append(train_loss_avg)
#             val_losses.append(val_loss_avg)
#             train_accuracies.append(train_accuracy)
#             val_accuracies.append(val_accuracy)
#
#             epoch_time = time.time() - start_time
#
#             print(f'Epoch [{epoch+1}/{num_epochs}], Time: {epoch_time:.2f}s')
#             print(f'  训练损失: {train_loss_avg:.4f}, 训练准确率: {train_accuracy:.4f}')
#             print(f'  验证损失: {val_loss_avg:.4f}, 验证准确率: {val_accuracy:.4f}')
#
#             # 早停法
#             if val_loss_avg < best_val_loss:
#                 best_val_loss = val_loss_avg
#                 patience_counter = 0
#                 # 保存最佳模型
#                 torch.save(self.model.state_dict(), 'best_model.pth')
#             else:
#                 patience_counter += 1
#                 if patience_counter >= patience:
#                     print(f"早停：验证损失在 {patience} 个epoch内没有改善")
#                     break
#
#         # 加载最佳模型
#         # 在代码中找到这行并修改
#         self.model.load_state_dict(torch.load('best_model.pth', weights_only=True))
#
#         return {
#             'train_losses': train_losses,
#             'val_losses': val_losses,
#             'train_accuracies': train_accuracies,
#             'val_accuracies': val_accuracies
#         }
#
#     def evaluate_model(self, test_loader):
#         """评估模型"""
#         self.model.eval()
#         all_predictions = []
#         all_targets = []
#
#         with torch.no_grad():
#             for sequences, masks, labels, lengths in test_loader:
#                 sequences = sequences.to(self.device)
#                 labels = labels.to(self.device)
#
#                 outputs = self.model(sequences)  # 不再传递lengths参数
#                 predictions = (outputs > 0.5).float()
#
#                 all_predictions.extend(predictions.cpu().numpy())
#                 all_targets.extend(labels.cpu().numpy())
#
#         # 计算评估指标
#         accuracy = accuracy_score(all_targets, all_predictions)
#         precision = precision_score(all_targets, all_predictions, zero_division=0)
#         recall = recall_score(all_targets, all_predictions, zero_division=0)
#         f1 = f1_score(all_targets, all_predictions, zero_division=0)
#
#         print("\n" + "="*50)
#         print("模型评估结果")
#         print("="*50)
#         print(f"准确率 (Accuracy): {accuracy:.4f}")
#         print(f"精确率 (Precision): {precision:.4f}")
#         print(f"召回率 (Recall): {recall:.4f}")
#         print(f"F1分数: {f1:.4f}")
#         print("\n分类报告:")
#         print(classification_report(all_targets, all_predictions, target_names=['负面', '正面']))
#
#         return {
#             'accuracy': accuracy,
#             'precision': precision,
#             'recall': recall,
#             'f1': f1,
#             'predictions': all_predictions,
#             'targets': all_targets
#         }

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
from collections import Counter
import time
import os
import json
import seaborn as sns
from itertools import product
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

class SentimentLSTM(nn.Module):
    """LSTM情感分析模型"""

    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 dropout, embedding_weights=None, train_embedding=True):
        super(SentimentLSTM, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # 嵌入层
        if embedding_weights is not None:
            self.embedding = nn.Embedding.from_pretrained(embedding_weights, freeze=not train_embedding)
        else:
            self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM层
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                           dropout=dropout, batch_first=True, bidirectional=False)

        # Dropout层
        self.dropout = nn.Dropout(dropout)

        # 全连接层
        self.fc = nn.Linear(hidden_dim, output_dim)

        # Sigmoid激活函数
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 嵌入层
        embedded = self.embedding(x)

        # LSTM层
        lstm_out, (hidden, cell) = self.lstm(embedded)

        # 取最后一个时间步的输出
        out = self.dropout(lstm_out[:, -1, :])

        # 全连接层
        out = self.fc(out)

        # Sigmoid激活
        out = self.sigmoid(out)

        return out.squeeze()



def plot_training_history(history, title="训练历史"):
    """绘制训练历史曲线"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # 损失曲线
    ax1.plot(history['train_losses'], label='训练损失', linewidth=2)
    ax1.plot(history['val_losses'], label='验证损失', linewidth=2)
    ax1.set_title(f'{title} - 损失曲线')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 准确率曲线
    ax2.plot(history['train_accuracies'], label='训练准确率', linewidth=2)
    ax2.plot(history['val_accuracies'], label='验证准确率', linewidth=2)
    ax2.set_title(f'{title} - 准确率曲线')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


使用设备: cuda


In [45]:
class SentimentAnalyzer:
    """完整的情感分析器类"""

    def __init__(self, device='cpu'):
        self.device = device
        self.vocab_size = 0
        self.model = None
        # 训练指标记录
        self.best_val_loss = float('inf')
        self.patience_counter = 0
        self.best_epoch = 0
        self.training_time = 0
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
        self.current_params = None
        self.metrics = {}

    def create_model(self, vocab_size, embedding_dim=100, hidden_dim=128, n_layers=2,
                    dropout=0.3, use_pretrained=True, glove_path="glove.6B.100d.txt", train_embedding=True):
        """创建LSTM模型"""
        self.vocab_size = vocab_size
        self.current_params = {
            'vocab_size': vocab_size,
            'embedding_dim': embedding_dim,
            'hidden_dim': hidden_dim,
            'n_layers': n_layers,
            'dropout': dropout,
            'use_pretrained': use_pretrained,
            'train_embedding': train_embedding
        }

        if use_pretrained and glove_path:
            embedding_weights = self._load_glove_embeddings(glove_path, embedding_dim, vocab_size)
            if embedding_weights is not None:
                self.model = SentimentLSTM(
                    vocab_size, embedding_dim, hidden_dim, 1, n_layers,
                    dropout, embedding_weights, train_embedding
                )
                print("使用预训练GloVe词向量的模型已创建")
            else:
                print("无法加载预训练词向量，使用随机初始化")
                use_pretrained = False

        if not use_pretrained:
            self.model = SentimentLSTM(
                vocab_size, embedding_dim, hidden_dim, 1, n_layers,
                dropout, None, True
            )
            print("使用随机初始化词向量的模型已创建")

        self.model = self.model.to(self.device)
        return self.model

    def _load_glove_embeddings(self, glove_path, embedding_dim, vocab_size):
        """加载预训练的GloVe词向量"""
        print("加载GloVe词向量...")

        # 创建随机初始化的词向量（由于缺乏词汇映射）
        embedding_weights = np.random.normal(scale=0.6, size=(vocab_size, embedding_dim))

        try:
            # 尝试加载GloVe文件
            glove_embeddings = {}
            with open(glove_path, 'r', encoding='utf-8') as f:
                for line in f:
                    values = line.split()
                    if len(values) < embedding_dim + 1:
                        continue
                    word = values[0]
                    vector = np.asarray(values[1:embedding_dim+1], dtype='float32')
                    glove_embeddings[word] = vector

            print(f"成功加载 {len(glove_embeddings)} 个GloVe词向量")
            print("注意: 由于缺乏索引到词的精确映射，使用随机初始化替代完整GloVe加载")

        except FileNotFoundError:
            print(f"GloVe文件未找到: {glove_path}")
            # print("请从 https://nlp.stanford.edu/projects/glove/ 下载GloVe词向量")
            return None

        return torch.tensor(embedding_weights, dtype=torch.float32)

    def train_model(self, train_loader, val_loader, num_epochs=10, learning_rate=0.001,
                   early_stopping_patience=3, save_path='best_model.pth'):
        """训练模型"""
        # 初始化训练指标
        self.best_val_loss = float('inf')
        self.patience_counter = 0
        self.best_epoch = 0
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

        criterion = nn.BCELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        print(f"开始训练，学习率: {learning_rate}, 早停耐心值: {early_stopping_patience}")
        start_time = time.time()

        for epoch in range(num_epochs):
            epoch_start_time = time.time()

            # 训练阶段
            self.model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0

            for batch_idx, (sequences, masks, labels, lengths) in enumerate(train_loader):
                sequences = sequences.to(self.device)
                labels = labels.to(self.device).float()

                optimizer.zero_grad()
                outputs = self.model(sequences)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                predicted = (outputs > 0.5).float()
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()

            # 计算训练准确率
            train_accuracy = train_correct / train_total
            avg_train_loss = train_loss / len(train_loader)

            # 验证阶段
            self.model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for sequences, masks, labels, lengths in val_loader:
                    sequences = sequences.to(self.device)
                    labels = labels.to(self.device).float()

                    outputs = self.model(sequences)
                    loss = criterion(outputs, labels)

                    val_loss += loss.item()
                    predicted = (outputs > 0.5).float()
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()

            val_accuracy = val_correct / val_total
            avg_val_loss = val_loss / len(val_loader)

            # 记录指标
            self.train_losses.append(avg_train_loss)
            self.val_losses.append(avg_val_loss)
            self.train_accuracies.append(train_accuracy)
            self.val_accuracies.append(val_accuracy)

            # 早停法
            if avg_val_loss < self.best_val_loss:
                self.best_val_loss = avg_val_loss
                self.best_epoch = epoch + 1
                self.patience_counter = 0
                torch.save(self.model.state_dict(), save_path)
                print(f"保存最佳模型，验证损失: {avg_val_loss:.4f}")
            else:
                self.patience_counter += 1

            epoch_time = time.time() - epoch_start_time

            # 打印进度
            print(f'Epoch [{epoch+1}/{num_epochs}], Time: {epoch_time:.2f}s')
            print(f'  训练损失: {avg_train_loss:.4f}, 训练准确率: {train_accuracy:.4f}')
            print(f'  验证损失: {avg_val_loss:.4f}, 验证准确率: {val_accuracy:.4f}')
            print(f'  最佳epoch: {self.best_epoch}, 早停计数: {self.patience_counter}/{early_stopping_patience}')

            # 检查早停
            if self.patience_counter >= early_stopping_patience:
                print(f"早停触发于第 {epoch+1} 个epoch")
                break

        # 记录总训练时间
        self.training_time = time.time() - start_time

        # 加载最佳模型
        self.model.load_state_dict(torch.load(save_path, weights_only=True))

        print(f"训练完成，总时间: {self.training_time:.2f}秒")
        return {
            'train_losses': self.train_losses,
            'val_losses': self.val_losses,
            'train_accuracies': self.train_accuracies,
            'val_accuracies': self.val_accuracies
        }

    def evaluate_model(self, test_loader):
        """评估模型性能"""
        self.model.eval()
        all_predictions = []
        all_targets = []
        all_probabilities = []

        with torch.no_grad():
            for sequences, masks, labels, lengths in test_loader:
                sequences = sequences.to(self.device)
                labels = labels.to(self.device)

                outputs = self.model(sequences)
                probabilities = outputs.cpu().numpy()
                predictions = (outputs > 0.5).float().cpu().numpy()

                all_predictions.extend(predictions)
                all_targets.extend(labels.cpu().numpy())
                all_probabilities.extend(probabilities)

        # 计算评估指标
        accuracy = accuracy_score(all_targets, all_predictions)
        precision = precision_score(all_targets, all_predictions, zero_division=0)
        recall = recall_score(all_targets, all_predictions, zero_division=0)
        f1 = f1_score(all_targets, all_predictions, zero_division=0)

        # 保存指标
        self.metrics = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'predictions': all_predictions,
            'targets': all_targets,
            'probabilities': all_probabilities
        }

        # 打印结果
        print("\n" + "="*60)
        print("模型评估结果")
        print("="*60)
        print(f"准确率 (Accuracy): {accuracy:.4f}")
        print(f"精确率 (Precision): {precision:.4f}")
        print(f"召回率 (Recall): {recall:.4f}")
        print(f"F1分数: {f1:.4f}")
        print("\n详细分类报告:")
        print(classification_report(all_targets, all_predictions,
                                  target_names=['负面', '正面'], digits=4))

        return self.metrics

    def predict_sentiment(self, text_sequences):
        """预测新文本的情感"""
        self.model.eval()
        with torch.no_grad():
            sequences = text_sequences.to(self.device)
            outputs = self.model(sequences)
            predictions = (outputs > 0.5).float()
            probabilities = outputs.cpu().numpy()
        return predictions.cpu().numpy(), probabilities

    def generate_report(self, report_dir='reports', filename='analysis_report.html'):
        """生成完整的分析报告"""
        import matplotlib.pyplot as plt
        os.makedirs(report_dir, exist_ok=True)

        # 收集报告数据
        report_data = {
            'best_val_loss': self.best_val_loss,
            'best_epoch': self.best_epoch,
            'training_time': self.training_time,
            'parameters': self.current_params,
            'metrics': self.metrics
        }

        # 生成HTML报告
        html_content = f"""
        <html>
        <head>
            <title>情感分析模型实验报告</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }}
                h1, h2, h3 {{ color: #2c3e50; }}
                .container {{ max-width: 1200px; margin: 0 auto; }}
                .metric-card {{ background: #f8f9fa; padding: 20px; margin: 10px; border-radius: 5px; }}
                .grid {{ display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px; }}
                table {{ width: 100%; border-collapse: collapse; margin: 20px 0; }}
                th, td {{ border: 1px solid #ddd; padding: 12px; text-align: left; }}
                th {{ background-color: #3498db; color: white; }}
                tr:nth-child(even) {{ background-color: #f2f2f2; }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>🧠 情感分析模型实验报告</h1>

                <h2>📊 实验概览</h2>
                <div class="grid">
                    <div class="metric-card">
                        <h3>最佳验证损失</h3>
                        <p>{report_data['best_val_loss']:.4f}</p>
                    </div>
                    <div class="metric-card">
                        <h3>最佳Epoch</h3>
                        <p>{report_data['best_epoch']}</p>
                    </div>
                    <div class="metric-card">
                        <h3>训练时间</h3>
                        <p>{report_data['training_time']:.2f}秒</p>
                    </div>
                    <div class="metric-card">
                        <h3>最终准确率</h3>
                        <p>{report_data['metrics'].get('accuracy', 0):.4f}</p>
                    </div>
                </div>

                <h2>⚙️ 模型参数</h2>
                <pre>{json.dumps(report_data['parameters'], indent=2, ensure_ascii=False)}</pre>

                <h2>📈 性能指标</h2>
                <table>
                    <tr><th>指标</th><th>数值</th></tr>
                    <tr><td>准确率 (Accuracy)</td><td>{report_data['metrics'].get('accuracy', 0):.4f}</td></tr>
                    <tr><td>精确率 (Precision)</td><td>{report_data['metrics'].get('precision', 0):.4f}</td></tr>
                    <tr><td>召回率 (Recall)</td><td>{report_data['metrics'].get('recall', 0):.4f}</td></tr>
                    <tr><td>F1分数</td><td>{report_data['metrics'].get('f1', 0):.4f}</td></tr>
                </table>

                <h2>📉 训练曲线</h2>
                <img src="training_curves.png" width="100%">
            </div>
        </body>
        </html>
        """

        # 保存HTML报告
        report_path = os.path.join(report_dir, filename)
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(html_content)

        # 生成训练曲线图
        self._plot_training_curves(report_dir)

        print(f"📄 分析报告已生成: {report_path}")
        return report_path

    def _plot_training_curves(self, save_dir):
        """绘制训练曲线"""
        plt.figure(figsize=(15, 5))

        # 损失曲线
        plt.subplot(1, 2, 1)
        plt.plot(self.train_losses, label='训练损失', linewidth=2)
        plt.plot(self.val_losses, label='验证损失', linewidth=2)
        plt.title('训练和验证损失曲线')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # 准确率曲线
        plt.subplot(1, 2, 2)
        plt.plot(self.train_accuracies, label='训练准确率', linewidth=2)
        plt.plot(self.val_accuracies, label='验证准确率', linewidth=2)
        plt.title('训练和验证准确率曲线')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, 'training_curves.png'), dpi=300, bbox_inches='tight')
        plt.close()


In [46]:
class ExperimentManager:
    """实验管理器 - 用于超参数调优和对比实验"""

    def __init__(self, device='cpu'):
        self.device = device
        self.experiment_results = []
        self.best_experiment = None

    def run_hyperparameter_search(self, train_loader, val_loader, test_loader, vocab_size,
                                 param_grid=None, max_experiments=20):
        """运行超参数搜索"""
        if param_grid is None:
            param_grid = {
                'learning_rate': [0.0001, 0.0005, 0.001],
                'batch_size': [32, 64],
                'dropout_rate': [0.2, 0.3, 0.5],
                'hidden_dim': [64, 128],
                'n_layers': [1, 2],
                'use_pretrained': [True, False]
            }

        print("开始超参数搜索实验...")
        experiment_count = 0

        # 生成参数组合
        param_combinations = self._generate_param_combinations(param_grid, max_experiments)

        for i, params in enumerate(param_combinations):
            if experiment_count >= max_experiments:
                break

            try:
                print(f"\n🔬 实验 {i+1}/{len(param_combinations)}")
                print(f"参数: {params}")

                # 创建新的数据加载器（针对不同的batch_size）
                batch_size = params['batch_size']
                current_train_loader = DataLoader(
                    train_loader.dataset, batch_size=batch_size, shuffle=True
                )
                current_val_loader = DataLoader(
                    val_loader.dataset, batch_size=batch_size, shuffle=False
                )
                current_test_loader = DataLoader(
                    test_loader.dataset, batch_size=batch_size, shuffle=False
                )

                # 创建和训练模型
                analyzer = SentimentAnalyzer(device=self.device)
                analyzer.create_model(
                    vocab_size=vocab_size,
                    embedding_dim=100,
                    hidden_dim=params['hidden_dim'],
                    n_layers=params['n_layers'],
                    dropout=params['dropout_rate'],
                    use_pretrained=params['use_pretrained']
                )

                # 训练模型
                history = analyzer.train_model(
                    current_train_loader, current_val_loader,
                    learning_rate=params['learning_rate'],
                    num_epochs=15,
                    early_stopping_patience=3
                )

                # 评估模型
                metrics = analyzer.evaluate_model(current_test_loader)

                # 记录实验结果
                experiment_result = {
                    'experiment_id': i + 1,
                    'parameters': params,
                    'metrics': metrics,
                    'history': history,
                    'training_time': analyzer.training_time,
                    'analyzer': analyzer
                }

                self.experiment_results.append(experiment_result)

                # 更新最佳实验
                if self.best_experiment is None or metrics['f1'] > self.best_experiment['metrics']['f1']:
                    self.best_experiment = experiment_result
                    print(f"🎯 新的最佳实验! F1分数: {metrics['f1']:.4f}")

                experiment_count += 1

            except Exception as e:
                print(f"❌ 实验 {i+1} 失败: {e}")
                continue

        print(f"\n✅ 超参数搜索完成! 共完成 {len(self.experiment_results)} 个实验")
        return self.experiment_results

    def _generate_param_combinations(self, param_grid, max_experiments):
        """生成参数组合"""
        keys = param_grid.keys()
        values = param_grid.values()

        combinations = []
        for combination in product(*values):
            param_dict = dict(zip(keys, combination))
            combinations.append(param_dict)

        # 如果组合太多，随机选择一部分
        if len(combinations) > max_experiments:
            indices = np.random.choice(len(combinations), max_experiments, replace=False)
            combinations = [combinations[i] for i in indices]

        return combinations

    def generate_comparison_report(self):
        """生成实验对比报告"""
        if not self.experiment_results:
            print("没有实验结果可比较")
            return

        # 创建对比DataFrame
        comparison_data = []
        for result in self.experiment_results:
            row = result['parameters'].copy()
            row.update(result['metrics'])
            row['training_time'] = result['training_time']
            row['experiment_id'] = result['experiment_id']
            comparison_data.append(row)

        df = pd.DataFrame(comparison_data)

        # 生成可视化报告
        self._plot_experiment_comparison(df)

        # 保存详细结果
        df.to_csv('experiment_comparison.csv', index=False, encoding='utf-8')

        print(f"\n📊 实验对比报告已生成")
        print(f"最佳实验ID: {self.best_experiment['experiment_id']}")
        print(f"最佳F1分数: {self.best_experiment['metrics']['f1']:.4f}")

        return df

    def _plot_experiment_comparison(self, df):
        """绘制实验对比图"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))

        # F1分数对比
        axes[0,0].bar(df['experiment_id'], df['f1'], alpha=0.7)
        axes[0,0].set_xlabel('实验ID')
        axes[0,0].set_ylabel('F1分数')
        axes[0,0].set_title('各实验F1分数对比')
        axes[0,0].grid(True, alpha=0.3)

        # 准确率对比
        axes[0,1].bar(df['experiment_id'], df['accuracy'], alpha=0.7, color='orange')
        axes[0,1].set_xlabel('实验ID')
        axes[0,1].set_ylabel('准确率')
        axes[0,1].set_title('各实验准确率对比')
        axes[0,1].grid(True, alpha=0.3)

        # 训练时间对比
        axes[1,0].bar(df['experiment_id'], df['training_time'], alpha=0.7, color='green')
        axes[1,0].set_xlabel('实验ID')
        axes[1,0].set_ylabel('训练时间(秒)')
        axes[1,0].set_title('训练时间对比')
        axes[1,0].grid(True, alpha=0.3)

        # 嵌入类型对比
        if 'use_pretrained' in df.columns:
            pretrained_mean = df[df['use_pretrained'] == True]['f1'].mean()
            random_mean = df[df['use_pretrained'] == False]['f1'].mean()
            axes[1,1].bar(['预训练', '随机'], [pretrained_mean, random_mean], alpha=0.7, color='red')
            axes[1,1].set_ylabel('平均F1分数')
            axes[1,1].set_title('嵌入类型性能对比')
            axes[1,1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig('experiment_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()


In [47]:
def plot_training_history(history, model_name):
    """绘制训练历史"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # 绘制损失曲线
    ax1.plot(history['train_losses'], label='训练损失')
    ax1.plot(history['val_losses'], label='验证损失')
    ax1.set_title(f'{model_name} - 损失曲线')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)

    # 绘制准确率曲线
    ax2.plot(history['train_accuracies'], label='训练准确率')
    ax2.plot(history['val_accuracies'], label='验证准确率')
    ax2.set_title(f'{model_name} - 准确率曲线')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.savefig(f'{model_name}_training_history.png', dpi=300, bbox_inches='tight')
    plt.show()


In [48]:
def simple_train_and_evaluate(analyzer, train_loader, val_loader, test_loader, vocab_size, use_pretrained=False):
    """简化的训练和评估流程"""
    print("\n" + "="*80)
    print(f"训练{'预训练嵌入' if use_pretrained else '随机嵌入'}模型")
    print("="*80)
    
    # 创建模型
    analyzer.create_model(
        vocab_size=vocab_size,
        hidden_dim=128,
        dropout=0.3,
        use_pretrained=use_pretrained
    )
    
    # 训练模型
    history = analyzer.train_model(
        train_loader, val_loader,
        learning_rate=0.001,
        num_epochs=5  # 减少epoch数以加快实验
    )
    
    # 评估模型
    results = analyzer.evaluate_model(test_loader)
    plot_training_history(history, f"{'预训练嵌入' if use_pretrained else '随机嵌入'}模型")
    
    return results


In [49]:
# def main():
#     """主函数"""
#     print("开始情感分析项目...")
#
#     # 加载数据
#     print("加载数据...")
#     try:
#         data = torch.load(r"all_data.pt")
#         print("数据加载成功!")
#
#         # 提取训练、验证和测试数据
#         train_sequences = data['train_sequences']
#         train_masks = data['train_masks']
#         train_labels = data['train_labels']
#         train_lengths = data['train_lengths']
#
#         val_sequences = data['val_sequences']
#         val_masks = data['val_masks']
#         val_labels = data['val_labels']
#         val_lengths = data['val_lengths']
#
#         test_sequences = data['test_sequences']
#         test_masks = data['test_masks']
#         test_labels = data['test_labels']
#         test_lengths = data['test_lengths']
#
#         print(f"训练集: {len(train_sequences)} 个样本")
#         print(f"验证集: {len(val_sequences)} 个样本")
#         print(f"测试集: {len(test_sequences)} 个样本")
#
#         # 计算词汇表大小（从序列中的最大索引+1）
#         vocab_size = max(
#             train_sequences.max().item(),
#             val_sequences.max().item(),
#             test_sequences.max().item()
#         ) + 1
#         print(f"词汇表大小: {vocab_size}")
#
#         # 创建数据集
#         train_dataset = TensorDataset(train_sequences, train_masks, train_labels, train_lengths)
#         val_dataset = TensorDataset(val_sequences, val_masks, val_labels, val_lengths)
#         test_dataset = TensorDataset(test_sequences, test_masks, test_labels, test_lengths)
#
#         # 创建数据加载器
#         batch_size = 32
#         train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#         val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
#         test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#
#         # ===== 在这里添加诊断代码 =====
#         print("\n=== 数据诊断信息 ===")
#         print(f"训练集大小: {len(train_dataset)}")
#         print(f"验证集大小: {len(val_dataset)}")
#         print(f"测试集大小: {len(test_dataset)}")
#
#         # 检查标签分布
#         print(f"训练集标签分布: {Counter(train_labels.numpy())}")
#         print(f"验证集标签分布: {Counter(val_labels.numpy())}")
#         print(f"测试集标签分布: {Counter(test_labels.numpy())}")
#
#         # 检查序列数据
#         print(f"序列形状: {train_sequences.shape}")
#         print(f"序列数据类型: {train_sequences.dtype}")
#         print(f"序列值范围: [{train_sequences.min()}, {train_sequences.max()}]")
#         print(f"样本标签示例: {train_labels[:10].numpy()}")
#
#         # 检查设备
#         print(f"训练设备: {device}")
#         print("====================\n")
#
#     except Exception as e:
#         print(f"数据加载失败: {e}")
#         return
#
#     # 其余的代码保持不变...
#
#     # 初始化分析器
#     analyzer = SentimentAnalyzer(device=device)
#
#     # 简化的训练流程 - 只训练随机嵌入模型
#     print("\n开始训练随机嵌入模型...")
#     random_results = simple_train_and_evaluate(analyzer, train_loader, val_loader, test_loader, vocab_size, use_pretrained=False)
#
#     # 尝试预训练嵌入模型（如果GloVe文件可用）
#     glove_path = "glove.6B.100d.txt"  # 请根据实际情况修改路径
#     try:
#         print("\n开始训练预训练嵌入模型...")
#         pretrained_results = simple_train_and_evaluate(analyzer, train_loader, val_loader, test_loader, vocab_size, use_pretrained=True)
#
#         # 比较结果
#         print("\n" + "="*80)
#         print("模型比较结果")
#         print("="*80)
#         print(f"随机嵌入模型 - 准确率: {random_results['accuracy']:.4f}, F1: {random_results['f1']:.4f}")
#         print(f"预训练嵌入模型 - 准确率: {pretrained_results['accuracy']:.4f}, F1: {pretrained_results['f1']:.4f}")
#
#         if pretrained_results['accuracy'] > random_results['accuracy']:
#             print("预训练嵌入模型表现更好!")
#         else:
#             print("随机嵌入模型表现更好!")
#
#     except Exception as e:
#         print(f"预训练嵌入模型训练失败: {e}")
#         print("请确保GloVe词向量文件可用，或使用随机嵌入模型")
#
#     print("\n项目完成!")
#
# if __name__ == "__main__":
#     main()

In [50]:
def main():
    """完整的演示流程"""
    print("="*80)
    print("🤖 情感分析系统 - 完整演示")
    print("="*80)

    try:
        # 加载数据
        print("📂 加载数据...")
        data = torch.load("all_data.pt")

        # 创建数据集
        train_dataset = TensorDataset(
            data['train_sequences'], data['train_masks'],
            data['train_labels'], data['train_lengths']
        )
        val_dataset = TensorDataset(
            data['val_sequences'], data['val_masks'],
            data['val_labels'], data['val_lengths']
        )
        test_dataset = TensorDataset(
            data['test_sequences'], data['test_masks'],
            data['test_labels'], data['test_lengths']
        )

        vocab_size = max(
            data['train_sequences'].max().item(),
            data['val_sequences'].max().item(),
            data['test_sequences'].max().item()
        ) + 1

        print(f"词汇表大小: {vocab_size}")
        print(f"训练集: {len(train_dataset)}个样本")
        print(f"验证集: {len(val_dataset)}个样本")
        print(f"测试集: {len(test_dataset)}个样本")

        # 创建基础数据加载器
        batch_size = 32
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # 选项1: 快速单个模型训练
        print("\n1. 🚀 快速训练单个模型...")
        analyzer = SentimentAnalyzer(device=device)
        analyzer.create_model(vocab_size=vocab_size, use_pretrained=True)

        history = analyzer.train_model(
            train_loader, val_loader,
            learning_rate=0.001, num_epochs=10
        )

        metrics = analyzer.evaluate_model(test_loader)
        analyzer.generate_report()

        # 选项2: 超参数搜索（可选）
        print("\n2. 🔍 是否进行超参数搜索? (y/n)")
        user_input = input().strip().lower()

        if user_input == 'y':
            print("开始超参数搜索...")
            experiment_manager = ExperimentManager(device=device)

            # 定义较小的参数网格用于快速演示
            quick_param_grid = {
                'learning_rate': [0.0005, 0.001],
                'batch_size': [32],
                'dropout_rate': [0.3, 0.5],
                'hidden_dim': [64, 128],
                'n_layers': [1, 2],
                'use_pretrained': [True,False]
            }

            results = experiment_manager.run_hyperparameter_search(
                train_loader, val_loader, test_loader, vocab_size,
                param_grid=quick_param_grid, max_experiments=8
            )

            experiment_manager.generate_comparison_report()

        print("\n✅ 程序执行完成!")

    except Exception as e:
        print(f"❌ 错误: {e}")
        print("请检查数据文件路径和格式")

if __name__ == "__main__":
    main()

🤖 情感分析系统 - 完整演示
📂 加载数据...
词汇表大小: 30000
训练集: 32000个样本
验证集: 8000个样本
测试集: 10000个样本

1. 🚀 快速训练单个模型...
加载GloVe词向量...
成功加载 400000 个GloVe词向量
注意: 由于缺乏索引到词的精确映射，使用随机初始化替代完整GloVe加载
使用预训练GloVe词向量的模型已创建
开始训练，学习率: 0.001, 早停耐心值: 3
保存最佳模型，验证损失: 0.6930
Epoch [1/10], Time: 12.73s
  训练损失: 0.6934, 训练准确率: 0.5024
  验证损失: 0.6930, 验证准确率: 0.5085
  最佳epoch: 1, 早停计数: 0/3
Epoch [2/10], Time: 12.33s
  训练损失: 0.6916, 训练准确率: 0.5067
  验证损失: 0.6935, 验证准确率: 0.5016
  最佳epoch: 1, 早停计数: 1/3
Epoch [3/10], Time: 12.36s
  训练损失: 0.6810, 训练准确率: 0.5198
  验证损失: 0.6991, 验证准确率: 0.5109
  最佳epoch: 1, 早停计数: 2/3
Epoch [4/10], Time: 12.28s
  训练损失: 0.6612, 训练准确率: 0.5338
  验证损失: 0.7191, 验证准确率: 0.5111
  最佳epoch: 1, 早停计数: 3/3
早停触发于第 4 个epoch
训练完成，总时间: 49.71秒

模型评估结果
准确率 (Accuracy): 0.5033
精确率 (Precision): 0.5295
召回率 (Recall): 0.0592
F1分数: 0.1065

详细分类报告:
              precision    recall  f1-score   support

          负面     0.5017    0.9474    0.6560      5000
          正面     0.5295    0.0592    0.1065      5000

    accuracy            